# 07 - Calibration and Threshold Tuning (V3)

- Calibrate finalists (Platt or Isotonic) with CV
- Tune threshold (maximize F1 or enforce recall floor)
- Evaluate on holdout test; save curves and confusion matrices


In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (confusion_matrix, f1_score, precision_recall_curve, 
                             classification_report, brier_score_loss)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt

INP = Path('../v3_data/employee_promotion_features.csv')
ART = Path('../v3_artifacts'); ART.mkdir(exist_ok=True)

TARGET = 'Promotion_Eligible'

df = pd.read_csv(INP)
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# For demonstration: use a GB finalist with nominal params (replace with tuned params if available)
base = GradientBoostingClassifier(random_state=42)
pipe = Pipeline([('pre', pre), ('model', base)])

cal = CalibratedClassifierCV(pipe, method='isotonic', cv=5)
cal.fit(X_train, y_train)
proba = cal.predict_proba(X_test)[:, 1]

# Threshold tuning: maximize F1
prec, rec, thr = precision_recall_curve(y_test, proba)
f1s = 2*prec*rec/(prec+rec + 1e-12)
best_idx = int(np.nanargmax(f1s))
best_thr = thr[max(best_idx-1, 0)] if best_idx < len(thr) else 0.5

pred = (proba >= best_thr).astype(int)
cm = confusion_matrix(y_test, pred)
print('Best threshold (F1):', best_thr)
print('Brier:', brier_score_loss(y_test, proba))
print('Confusion Matrix:\n', cm)
print('\nReport:\n', classification_report(y_test, pred, digits=3))

# Save calibration curve
prob_true, prob_pred = calibration_curve(y_test, proba, n_bins=10)
plt.figure(figsize=(4,4))
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1],'--',color='gray')
plt.title('Calibration Curve (Isotonic)')
plt.xlabel('Predicted probability')
plt.ylabel('True fraction positive')
plt.tight_layout()
plt.savefig(ART / 'calibration_curve.png', dpi=160)
plt.close()

# Save threshold
with open(ART / 'best_threshold.json', 'w') as f:
    json.dump({'best_threshold_f1': float(best_thr)}, f, indent=2)

